In [65]:
from DB import DataFrameHandler
from DB import AIModelHandler
import pandas as pd
import pickle, time
import os

pd.options.mode.chained_assignment = None

def existing_files():
    maindirectory_path = os.path.join('DB', 'logo')
    file_list = os.listdir(maindirectory_path)
    result = pd.DataFrame({"ISIN":file_list})
    result["ISIN"] = result["ISIN"].str.replace(".png",'')
    result.sort_values("ISIN", inplace=True)
    return result.reset_index(drop=True)

In [79]:
logos = existing_files()
logos

,ISIN
0,000020
1,000040
2,000050
3,000070
4,000075
...,...
2611,950170
2612,950190
2613,950200
2614,950210


In [80]:
stocks = pd.DataFrame({"ISIN":AIModelHandler.selectAllStock()})
stocks

,ISIN
0,000020
1,000040
2,000050
3,000070
4,000075
...,...
2611,950160
2612,950170
2613,950190
2614,950200


In [81]:
df = pd.merge(stocks, logos, how='outer', indicator=True)
stocks_only = df.query("_merge=='left_only'")
logos_only = df.query("_merge=='right_only'")

In [82]:
df

,ISIN,_merge
0,000020,both
1,000040,both
2,000050,both
3,000070,both
4,000075,both
...,...,...
2611,950160,both
2612,950170,both
2613,950190,both
2614,950200,both


In [83]:
stocks_only

,ISIN,_merge


In [84]:
logos_only

,ISIN,_merge


In [72]:
def delete_files(stocks):
    stocks = stocks['ISIN'].to_list()
    maindirectory_path = os.path.join('DB', 'logo')
    for stock in stocks:
        subdirectory_path = os.path.join(maindirectory_path, f'{stock}.png')
        if os.path.exists(subdirectory_path):
            os.remove(subdirectory_path)
            print('성공~~')
        else:
            print('비상~~')

In [77]:
import requests
import os
from bs4 import BeautifulSoup

data = stocks_only['ISIN'].to_list()
save_directory = r"/Users/Hong-YoonKi/HANAKOPO/FINAL/Flask/app/DB/logo"

failed = []

def download_image(url, save_path):
    img_response = requests.get(url, stream=True)
    img_response.raise_for_status()
    with open(save_path, "wb") as f:
        for chunk in img_response.iter_content(chunk_size=5000):
            f.write(chunk)

success = 0
for isin in data:

    try:
        img_url = f"https://file.alphasquare.co.kr/media/images/stock_logo/kr/{isin}.png"
        save_path = os.path.join(save_directory, f"{isin}.png")
        download_image(img_url, save_path)
        success += 1

    except Exception as e:
        failed.append(isin)
        print(f"오류 발생({isin}): {e}")
        print(f"이미지를 찾을 수 없음 ({isin})")

print("크롤링이 완료되었습니다. : " + str(success) + "개")


크롤링이 완료되었습니다. : 20개


In [78]:
failed

[]

In [73]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import time
import os

data = stocks_only['ISIN'].to_list()
save_directory = r"/Users/Hong-YoonKi/HANAKOPO/FINAL/Flask/app/DB/logo"

if not os.path.exists(save_directory):
    os.makedirs(save_directory)

def download_image(url, save_path):
    img_response = requests.get(url, stream=True)
    img_response.raise_for_status()
    with open(save_path, "wb") as f:
        for chunk in img_response.iter_content(chunk_size=5000):
            f.write(chunk)
    print("성공ㅎ")

success = 0
failed = []

driver = webdriver.Chrome()

for isin in data:
    url = f"https://www.alphasquare.co.kr/home/special-factor?code={isin}"
    page_source = driver.page_source
    driver.get(url)
    time.sleep(3)

    try:
        img_tag = driver.find_element(
            By.XPATH, '//*[@id="main-chart"]/div[1]/div[2]/div[2]/div/div/img'
        )

        if img_tag:
            img_url = img_tag.get_attribute("src")
            save_path = os.path.join(save_directory, f"{isin}.png")
            download_image(img_url, save_path)
            success += 1

    except Exception as e:
        print(f"오류 발생({isin}): {e}")
        failed.append(isin)

driver.quit()
print("크롤링이 완료되었습니다. : " + str(success) + "개")
print(f"실패({len(failed)}) : {', '.join(failed)}")